<a href="https://colab.research.google.com/github/b2200765008/Detecting-Endangered-Species-from-Camera-Trap/blob/main/models-CNN-SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import cv2

import cv2
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import confusion_matrix, classification_report

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, BatchNormalization, Conv2D, Dense, Dropout, Flatten, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras import layers, models, losses
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
! unzip /content/drive/MyDrive/species_resized.zip

Görüntülenen çıkış son 5000 satıra kısaltıldı.
  inflating: Tapirus_bairdii_resized/image 2669.jpg  
  inflating: Tapirus_bairdii_resized/image 267.jpg  
  inflating: Tapirus_bairdii_resized/image 2670.jpg  
  inflating: Tapirus_bairdii_resized/image 2671.jpg  
  inflating: Tapirus_bairdii_resized/image 2672.jpg  
  inflating: Tapirus_bairdii_resized/image 2673.jpg  
  inflating: Tapirus_bairdii_resized/image 2674.jpg  
  inflating: Tapirus_bairdii_resized/image 2675.jpg  
  inflating: Tapirus_bairdii_resized/image 2676.jpg  
  inflating: Tapirus_bairdii_resized/image 2677.jpg  
  inflating: Tapirus_bairdii_resized/image 2678.jpg  
  inflating: Tapirus_bairdii_resized/image 2679.jpg  
  inflating: Tapirus_bairdii_resized/image 268.jpg  
  inflating: Tapirus_bairdii_resized/image 2680.jpg  
  inflating: Tapirus_bairdii_resized/image 2681.jpg  
  inflating: Tapirus_bairdii_resized/image 2682.jpg  
  inflating: Tapirus_bairdii_resized/image 2683.jpg  
  inflating: Tapirus_bairdii_resized/

In [ ]:
def measure_brightness(image):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    brightness = np.mean(gray_image)
    return brightness

def custom_preprocessing(image):
    # Measure brightness
    brightness_level = measure_brightness(image)

    # Adjust brightness only for night images (customize this condition)
    if brightness_level < 0.5:
        # Apply brightness adjustment (customize the factor)
        brightness_factor = 2.0
        image = cv2.convertScaleAbs(image, alpha=brightness_factor, beta=0)

    return image

In [ ]:
train_datagen = ImageDataGenerator(
    rescale = 1./255,
    # rotation_range = 15,
    # brightness_range = (0.7,1.3),
    cval=0,
    preprocessing_function=custom_preprocessing,
    validation_split=0.2,
    dtype = 'float32'
    )

val_datagen = ImageDataGenerator(
    rescale = 1./255,
    validation_split=0.2,
    dtype='float32'
    )

train_set = train_datagen.flow_from_directory(
                '/content/images',
                target_size = (128,128),
                batch_size =32,
                class_mode = 'categorical',
                shuffle = True,
                seed=42,
                subset='training',)



val_set = val_datagen.flow_from_directory(
                '/content/images',
                target_size = (128,128),
                batch_size =32,
                class_mode = 'categorical',
                subset='validation',
                shuffle=False,
                seed=6,
                )

Found 28518 images belonging to 10 classes.
Found 7123 images belonging to 10 classes.


In [ ]:
def create_model():
    model = Sequential([
    #     Conv2D(filters=128, kernel_size=(3, 3), padding='valid', input_shape=(228,228, 3)),
    #     Activation('relu'),
    #     MaxPooling2D(pool_size=(2, 2)),
    #     BatchNormalization(),

        Conv2D(filters=64, kernel_size=(3, 3), padding='valid', input_shape=(128,128, 3)),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),

        Conv2D(filters=32, kernel_size=(3, 3), padding='valid', kernel_regularizer=l2(0.001)),
        Activation('relu'),
        MaxPooling2D(pool_size=(2, 2)),
        BatchNormalization(),

        # Conv2D(filters=16, kernel_size=(3, 3), padding='valid', kernel_regularizer=l2(0.005)),
        # Activation('relu'),
        # MaxPooling2D(pool_size=(2, 2)),
        # BatchNormalization(),

        Flatten(),

        Dense(units=256, activation='relu'),
        Dropout(0.5),
        Dense(units=10, activation='softmax')
    ])

    return model

In [ ]:
model = create_model()
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 126, 126, 64)      1792      
                                                                 
 activation_4 (Activation)   (None, 126, 126, 64)      0         
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 63, 63, 64)        0         
 g2D)                                                            
                                                                 
 batch_normalization_4 (Bat  (None, 63, 63, 64)        256       
 chNormalization)                                                
                                                                 
 conv2d_5 (Conv2D)           (None, 61, 61, 32)        18464     
                                                                 
 activation_5 (Activation)   (None, 61, 61, 32)       

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=np.sqrt(0.1), patience=3)
optimizer = Adam(learning_rate=0.0005)
model.compile(optimizer=optimizer, loss=CategoricalCrossentropy(), metrics=['accuracy'])

In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
history = model.fit(train_set, epochs=20, validation_data=val_set,
                       verbose=2,
                       callbacks=[reduce_lr])

Epoch 1/20
446/446 - 47s - loss: 0.9143 - accuracy: 0.7910 - val_loss: 1.9070 - val_accuracy: 0.4648 - lr: 5.0000e-04 - 47s/epoch - 106ms/step
Epoch 2/20
446/446 - 47s - loss: 0.2700 - accuracy: 0.9209 - val_loss: 1.6892 - val_accuracy: 0.7173 - lr: 5.0000e-04 - 47s/epoch - 106ms/step
Epoch 3/20
446/446 - 44s - loss: 0.2032 - accuracy: 0.9445 - val_loss: 1.3187 - val_accuracy: 0.7062 - lr: 5.0000e-04 - 44s/epoch - 100ms/step
Epoch 4/20
446/446 - 44s - loss: 0.1725 - accuracy: 0.9565 - val_loss: 1.6158 - val_accuracy: 0.7166 - lr: 5.0000e-04 - 44s/epoch - 100ms/step
Epoch 5/20
446/446 - 42s - loss: 0.1307 - accuracy: 0.9686 - val_loss: 2.0332 - val_accuracy: 0.6827 - lr: 5.0000e-04 - 42s/epoch - 95ms/step
Epoch 6/20
446/446 - 51s - loss: 0.1325 - accuracy: 0.9698 - val_loss: 1.2988 - val_accuracy: 0.7180 - lr: 5.0000e-04 - 51s/epoch - 114ms/step
Epoch 7/20
446/446 - 46s - loss: 0.1187 - accuracy: 0.9738 - val_loss: 2.7623 - val_accuracy: 0.7031 - lr: 5.0000e-04 - 46s/epoch - 102ms/step


KeyboardInterrupt: ignored

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

# Load the VGG16 model
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Create a new model with custom dense layers
model = Sequential()
model.add(base_model)
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dense(4096, activation='relu'))
model.add(Dense(10, activation='softmax'))  # Replace num_classes with your actual number of classes

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Assuming you have a train_generator and val_generator from your ImageDataGenerator
history = model.fit(train_set, epochs=20, validation_data=val_set, verbose=2)


Epoch 1/20


Exception ignored in: <function _xla_gc_callback at 0x7e9c889cd870>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 101, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


502/502 - 249s - loss: 2.5501 - accuracy: 0.1197 - val_loss: 2.2973 - val_accuracy: 0.1216 - 249s/epoch - 495ms/step
Epoch 2/20
502/502 - 219s - loss: 2.2976 - accuracy: 0.1218 - val_loss: 2.2971 - val_accuracy: 0.1216 - 219s/epoch - 436ms/step
Epoch 3/20
502/502 - 216s - loss: 2.2974 - accuracy: 0.1218 - val_loss: 2.2970 - val_accuracy: 0.1216 - 216s/epoch - 431ms/step
Epoch 4/20
502/502 - 215s - loss: 2.2973 - accuracy: 0.1218 - val_loss: 2.2971 - val_accuracy: 0.1216 - 215s/epoch - 428ms/step
Epoch 5/20
502/502 - 216s - loss: 2.2972 - accuracy: 0.1218 - val_loss: 2.2971 - val_accuracy: 0.1216 - 216s/epoch - 431ms/step
Epoch 6/20
502/502 - 213s - loss: 2.2972 - accuracy: 0.1218 - val_loss: 2.2970 - val_accuracy: 0.1216 - 213s/epoch - 424ms/step
Epoch 7/20
502/502 - 217s - loss: 2.2973 - accuracy: 0.1218 - val_loss: 2.2971 - val_accuracy: 0.1216 - 217s/epoch - 433ms/step
Epoch 8/20


InvalidArgumentError: ignored

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Load the ResNet50 model
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Create a new model with custom dense layers
model = Sequential()
model.add(base_model)
model.add(Flatten())
# model.add(Dense(2048, activation='relu'))
# model.add(Dropout(0.5))  # Add dropout for regularization
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))  # Add dropout for regularization
model.add(Dense(10, activation='softmax'))  # Replace 10 with your actual number of classes

# Compile the model
optimizer = Adam(learning_rate=0.0005)  # Adjust the learning rate as needed
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Assuming you have a train_generator and val_generator from your ImageDataGenerator
history = model.fit(
    train_set,
    epochs=20,
    validation_data=val_set,
    verbose=2,
    callbacks=[
        EarlyStopping(patience=5, restore_best_weights=True),  # Add early stopping
        ModelCheckpoint('best_model.h5', save_best_only=True),  # Save the best model
    ]
)


Epoch 1/20
446/446 - 156s - loss: 0.4348 - accuracy: 0.9088 - val_loss: 2.4092 - val_accuracy: 0.1529 - 156s/epoch - 351ms/step
Epoch 2/20
446/446 - 130s - loss: 0.1771 - accuracy: 0.9567 - val_loss: 9026.8223 - val_accuracy: 0.1115 - 130s/epoch - 291ms/step
Epoch 3/20
446/446 - 129s - loss: 0.8284 - accuracy: 0.8962 - val_loss: 2.0761 - val_accuracy: 0.5656 - 129s/epoch - 289ms/step
Epoch 4/20
446/446 - 129s - loss: 0.1829 - accuracy: 0.9456 - val_loss: 0.8264 - val_accuracy: 0.7988 - 129s/epoch - 288ms/step
Epoch 5/20
446/446 - 121s - loss: 0.0818 - accuracy: 0.9765 - val_loss: 1.0509 - val_accuracy: 0.8077 - 121s/epoch - 270ms/step
Epoch 6/20
446/446 - 128s - loss: 0.0575 - accuracy: 0.9820 - val_loss: 0.8009 - val_accuracy: 0.7915 - 128s/epoch - 287ms/step
Epoch 7/20
446/446 - 119s - loss: 0.0554 - accuracy: 0.9834 - val_loss: 1.1519 - val_accuracy: 0.7905 - 119s/epoch - 268ms/step
Epoch 8/20
446/446 - 119s - loss: 0.0685 - accuracy: 0.9824 - val_loss: 0.9419 - val_accuracy: 0.7915

In [ ]:
from tensorflow.keras.applications import EfficientNetB7

# Load the EfficientNetB7 model
base_model = EfficientNetB7(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Create a new model with custom dense layers
model = Sequential()
model.add(base_model)
model.add(Flatten())
# model.add(Dense(2048, activation='relu'))
# model.add(Dropout(0.5))  # Add dropout for regularization
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))  # Add dropout for regularization
model.add(Dense(10, activation='softmax'))  # Replace 10 with your actual number of classes

# Compile the model
optimizer = Adam(learning_rate=0.00001)  # Adjust the learning rate as needed
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Assuming you have a train_generator and val_generator from your ImageDataGenerator
history = model.fit(
    train_set,
    epochs=20,
    validation_data=val_set,
    verbose=2,
    callbacks=[
        EarlyStopping(patience=5, restore_best_weights=True),  # Add early stopping
          # Save the best model
    ]
)


Epoch 1/20
892/892 - 710s - loss: 0.6457 - accuracy: 0.7852 - val_loss: 0.6720 - val_accuracy: 0.7747 - 710s/epoch - 796ms/step
Epoch 2/20
892/892 - 539s - loss: 0.1824 - accuracy: 0.9399 - val_loss: 0.5463 - val_accuracy: 0.8342 - 539s/epoch - 605ms/step
Epoch 3/20
892/892 - 526s - loss: 0.0957 - accuracy: 0.9686 - val_loss: 0.6382 - val_accuracy: 0.8133 - 526s/epoch - 590ms/step
Epoch 4/20
892/892 - 526s - loss: 0.0562 - accuracy: 0.9816 - val_loss: 0.5661 - val_accuracy: 0.8526 - 526s/epoch - 589ms/step
Epoch 5/20
892/892 - 526s - loss: 0.0367 - accuracy: 0.9883 - val_loss: 0.6642 - val_accuracy: 0.8144 - 526s/epoch - 590ms/step
Epoch 6/20
892/892 - 537s - loss: 0.0230 - accuracy: 0.9926 - val_loss: 0.6329 - val_accuracy: 0.8544 - 537s/epoch - 602ms/step
Epoch 7/20
892/892 - 539s - loss: 0.0182 - accuracy: 0.9942 - val_loss: 0.6653 - val_accuracy: 0.8407 - 539s/epoch - 604ms/step


In [4]:
import pandas as pd
import os
from skimage.transform import resize
from skimage.io import imread
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix
import cv2

In [5]:
datadir='/content/images/'
categories = [d for d in os.listdir(datadir) if os.path.isdir(os.path.join(datadir, d))]
flat_data_arr=[] #input array
target_arr=[] #output array
batch_size = 32  # Adjust the batch size based on your available memory
for i in categories:
    if(categories.index(i) == 4):
      break
    print(f'loading... category : {i}')
    path = os.path.join(datadir, i)
    for batch_start in range(0, len(os.listdir(path)), batch_size):
        batch_end = batch_start + batch_size
        batch_images = os.listdir(path)[batch_start:batch_end]
        for img in batch_images:
            img_array = imread(os.path.join(path, img))
            img_resized = resize(img_array, (112, 112))
            flat_data_arr.append(img_resized.flatten())
            target_arr.append(categories.index(i))
            del img_array
            del img_resized





loading... category : loxodonta_africana_resized
loading... category : Tapirus_bairdii_resized
loading... category : giraffa_camelopardalis_resized
loading... category : crax_rubra_resized


In [6]:
for i in categories:
    if(categories.index(i) < 4):
      continue
    elif(categories.index(i) ==8):
      break
    print(f'loading... category : {i}')
    path = os.path.join(datadir, i)
    for batch_start in range(0, len(os.listdir(path)), batch_size):
        batch_end = batch_start + batch_size
        batch_images = os.listdir(path)[batch_start:batch_end]
        for img in batch_images:
            img_array = imread(os.path.join(path, img))
            img_resized = resize(img_array, (112, 112))
            flat_data_arr.append(img_resized.flatten())
            target_arr.append(categories.index(i))
            del img_array
            del img_resized

loading... category : capra_aegagrus_resized
loading... category : panthera_onca_resized
loading... category : equus_quagga_resized
loading... category : macaca_nemestrina_resized


In [ ]:
for i in categories:
    if(categories.index(i) < 8):
      continue
    print(f'loading... category : {i}')
    path = os.path.join(datadir, i)
    for batch_start in range(0, len(os.listdir(path)), batch_size):
        batch_end = batch_start + batch_size
        batch_images = os.listdir(path)[batch_start:batch_end]
        for img in batch_images:
            img_array = imread(os.path.join(path, img))
            img_resized = resize(img_array, (112, 112))
            flat_data_arr.append(img_resized.flatten())
            target_arr.append(categories.index(i))
            del img_array
            del img_resized

loading... category : Tayassu_pecari_resized


In [ ]:
datadir='/content/images/'
categories = [d for d in os.listdir(datadir) if os.path.isdir(os.path.join(datadir, d))]
# Function to extract histogram features from an image
def extract_histogram(image):
    hist = []
    for i in range(3):  # Assuming RGB images
        channel_hist = cv2.calcHist([image], [i], None, [256], [0, 256])
        hist.extend(channel_hist.flatten())
    return np.array(hist)

# Extract features from all images
features = []
labels = []
for i in categories:
  path = os.path.join(datadir, i)
  for img_file in os.listdir(path):
    img = cv2.imread(os.path.join(path, img_file))
    img = cv2.resize(img, (224, 224))  # adjust the size as needed
    hist_features = extract_histogram(img)
    features.append(hist_features)
    labels.append(categories.index(i))

# Convert the list to a NumPy array
flat_data_arr = np.array(features)
target_arr = np.array(labels)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(flat_data_arr, target_arr, test_size=0.2, random_state=42)
# from sklearn.preprocessing import StandardScaler
# from sklearn.decomposition import PCA

# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)

# # Apply PCA
# pca = PCA(n_components= 250)
# X_train_pca = pca.fit_transform(X_train)

In [ ]:
print(np.shape(X_train))
print(np.shape(y_train))

In [ ]:
svm_model = svm.SVC(kernel='linear', C=1)
svm_model.fit(X_train, y_train)

In [ ]:
predictions = svm_model.predict(X_test)
print(classification_report(y_test, predictions))

In [ ]:
# defining parameter range
param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['linear']}

grid = GridSearchCV(svm.SVC(), param_grid, refit = True, verbose = 3)

# fitting the model for grid search
grid.fit(X_train, y_train)

[5, 8, 4, 2, 1, 9, 0, 7, 8, 7, 2, 7, 1, 6, 8, 2, 0, 1, 8, 0, 1, 3, 9, 2, 9, 0, 8, 2, 9, 7, 3, 6, 6, 4, 7, 1, 7, 7, 2, 6, 4, 9, 3, 4, 1, 7, 2, 4, 9, 9, 6, 7, 2, 7, 9, 7, 6, 3, 9, 3, 8, 4, 7, 3, 5, 5, 2, 7, 3, 3, 1, 9, 8, 4, 6, 0, 6, 6, 6, 2, 2, 3, 6, 8, 8, 9, 5, 7, 5, 5, 3, 7, 6, 2, 1, 9, 5, 8, 9, 0, 0, 4, 5, 4, 6, 8, 0, 0, 7, 6, 4, 4, 5, 1, 3, 9, 5, 0, 1, 5, 9, 2, 6, 1, 0, 1, 7, 6, 4, 7, 9, 6, 0, 4, 3, 4, 4, 4, 1, 5, 4, 5, 1, 7, 4, 4, 0, 6, 1, 1, 7, 1, 2, 0, 7, 1, 9, 3, 1, 3, 6, 9, 1, 1, 8, 7, 9, 4, 8, 2, 1, 5, 2, 3, 1, 3, 8, 3, 9, 7, 5, 1, 3, 9, 9, 4, 3, 5, 0, 4, 4, 2, 0, 5, 8, 4, 6, 6, 4, 6, 1, 8, 9, 9, 6, 8, 8, 7, 6, 6, 2, 1, 5, 8, 6, 7, 5, 5, 8, 2, 9, 1, 4, 2, 4, 5, 2, 5, 5, 9, 2, 8, 1, 5, 7, 7, 5, 8, 9, 5, 7, 8, 0, 5, 7, 4, 7, 0, 3, 7, 3, 2, 5, 4, 1, 3, 9, 6, 5, 0, 6, 8, 8, 4, 6, 5, 8, 9, 8, 1, 4, 8, 8, 7, 7, 6, 9, 6, 5, 5, 9, 7, 0, 5, 7, 1, 3, 2, 6, 4, 7, 1, 7, 3, 3, 9, 9, 3, 1, 2, 6, 7, 1, 8, 3, 5, 0, 4, 8, 6, 6, 3, 4, 4, 7, 3, 9, 4, 5, 3, 7, 7, 3, 9, 4, 9, 7, 9, 6, 0, 6, 6, 7, 

In [ ]:
#input size  (64,64) - accuracy 0.38 , precision 0.52, recall 0.39